In [ ]:
import pandas as pd
import numpy as np

In [ ]:
match = pd.read_csv('IPL_Matches_2008_2022.csv')
delivery = pd.read_csv('IPL_Ball_by_Ball_2008_2022.csv')

In [ ]:
total_score_df = delivery.groupby(['ID', 'innings']).sum()['total_run'].reset_index()

In [ ]:
total_score_df = total_score_df[total_score_df['innings'] == 1]
total_score_df.head()

In [ ]:
match_df = match.merge(total_score_df[['ID', 'total_run']], on='ID')

In [ ]:
match_df.head(2)

In [ ]:
teams = [
    'Chennai Super Kings',
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Punjab Kings',
    'Rajasthan Royals',
    'Delhi Capitals',
    'Gujarat Titans',
    'Lucknow Super Giants'
]

match_df.replace('Gujarat Lions', 'Gujarat Titans', inplace=True)
match_df.replace('Delhi Daredevils', 'Delhi Capitals', inplace=True)
match_df.replace('Pune Warriors ', 'Lucknow Super Giants', inplace=True)
match_df.replace('Deccan Chargers', 'Sunrisers Hyderabad', inplace=True)
match_df.replace('Rising Pune Supergiant', 'Lucknow Super Giants', inplace=True)
match_df.replace('Kings XI Punjab', 'Punjab Kings', inplace=True)

In [ ]:
match_df = match_df[match_df['Team1'].isin(teams)]
match_df = match_df[match_df['Team2'].isin(teams)]

match_df['Team1'].unique()

In [ ]:
match_df['method'].unique()

In [ ]:
match_df = match_df[match_df['method'] != 'D/L']

In [ ]:
fdf= match_df

In [ ]:
fdf = match_df[['ID', 'City', 'WinningTeam', 'total_run' , 'Team1' , 'Team2']]

In [ ]:
fdf

In [ ]:
delivery_df =  fdf.merge(delivery, on='ID')

In [ ]:
delivery_df = delivery_df[delivery_df['innings'] == 2]

In [ ]:
delivery_df['current_score']= delivery_df.groupby('ID').cumsum()['total_run_y']

In [ ]:
delivery_df['runs_left']= delivery_df['total_run_x'] - delivery_df['current_score']

In [ ]:
delivery_df.head(6)

In [ ]:
delivery_df['balls_left']= 120 - (delivery_df['overs']*6 + delivery_df['ballnumber'])

In [ ]:
delivery_df['player_out'] = delivery_df['player_out'].fillna("0")

In [ ]:
delivery_df['player_out'] = delivery_df['player_out'].apply(lambda x:x if x=='0' else "1")

In [ ]:
delivery_df.tail(5)

In [ ]:
delivery_df['player_out'] = delivery_df['player_out'].astype('int')
wicket = delivery_df.groupby('ID').cumsum()['player_out'].values
delivery_df['wicket'] = 10-wicket 

In [ ]:
delivery_df['crr']=  (delivery_df['current_score']*6) / (120-delivery_df['balls_left'])
delivery_df['rrr']=  (delivery_df['runs_left']*6)/(delivery_df['balls_left'])

In [ ]:
delivery_df.head(10)

In [ ]:
def result(row):
    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

In [ ]:
delivery_df['result'] = delivery_df.apply(result, axis=1)
delivery_df.tail(1)

In [ ]:
d = delivery_df

Team1_mask = d['BattingTeam'] == d['Team1']

Team2_mask = d['BattingTeam'] == d['Team2']

d['BowlingTeam'] = np.where(Team1_mask, d['Team2'], d['Team1'])

In [ ]:
final_df = d[['BattingTeam', 'BowlingTeam', 'City', 'runs_left', 'balls_left', 'wicket', 'total_run_x', 'crr', 'rrr', 'result']]

In [ ]:
final_df

In [ ]:
teams = [
    'Chennai Super Kings',
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Punjab Kings',
    'Rajasthan Royals',
    'Delhi Capitals',
    'Gujarat Titans',
    'Lucknow Super Giants'
]

final_df.replace('Gujarat Lions', 'Gujarat Titans', inplace=True)
final_df.replace('Delhi Daredevils', 'Delhi Capitals', inplace=True)
final_df.replace('Pune Warriors ', 'Lucknow Super Giants', inplace=True)
final_df.replace('Deccan Chargers', 'Sunrisers Hyderabad', inplace=True)
final_df.replace('Rising Pune Supergiant', 'Lucknow Super Giants', inplace=True)
final_df.replace('Kings XI Punjab', 'Punjab Kings', inplace=True)

In [ ]:
final_df = final_df[final_df['BattingTeam'].isin(teams)]
final_df = final_df[final_df['BowlingTeam'].isin(teams)]

final_df['BattingTeam'].unique()

In [ ]:
final_df.sample()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
final_df.shape

In [ ]:
final_df = final_df[final_df['balls_left'] != 0]

In [ ]:
x = final_df.iloc[:, :-1]
y = final_df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
x_train.info()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse=False, drop='first'),['BattingTeam', 'BowlingTeam', 'City'])
]
,remainder='passthrough')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('step1', trf),
    ('step2', LogisticRegression(solver='liblinear'))
])

pipe.fit(x_train, y_train)


In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
pipe.predict_proba(x_test)[10]